<a href="https://colab.research.google.com/github/schmellerl/gradient_flows_order_parameters_mechanics/blob/main/Three_Phase_Stokes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl
    import dolfin
else:
    try:
        import ufl
        import dolfin
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import ufl
        import dolfin

In [2]:
from google.colab import drive
drive.mount('/content/drive')
filepath = '/content/drive/MyDrive/ColabNumerics/Three_Phase_StokeS/'

Mounted at /content/drive


In [ ]:
!git clone https://github.com/schmellerl/gradient_flows_order_parameters_mechanics.git
from gradient_flows_order_parameters_mechanics.colab.Data.refine_mesh import refine_mesh
from gradient_flows_order_parameters_mechanics.colab.Data.model_parameters import *

from gradient_flows_order_parameters_mechanics.colab.Data.postprocessing    import *

<h1>Gradient flows for coupling order parameters and mechanics</h1>

FENICS implementation of the examples from

Schmeller, L. & Peschka, D. (2021). arXiv:XXXX.YYYY


DOI:

<h3>General coupled gradient flow evolution</h3>

For a state variable $q=(u,\psi)$ with displacement $u:\Omega\to\mathbb{R}^d$ and phase field $\psi:\Omega\to\mathbb{R}^N$ we consider the following free energy 
\begin{align}
    \mathscr{F}(q) 
    = & 
    \int_{{\Omega}}W_{\rm elast}(F_e,\psi){\rm d}x
    + \int_{{\Omega}}W_{\rm phase}(\psi,\nabla\psi,F)
    {\rm d}x
\end{align}

for deformation gradient $F=\mathbb{I}_d+\nabla u=F_e F_p$ and with given plastic strain $F_p=F_p(\psi)$ and $F_e=FF_p^{-1}$.

We consider a Neo-Hookean elastic energy density 
\begin{align}
   W_{\rm elast}(F_e,\psi) = \frac{G}{2}\left(\text{tr}(F_e^\top F_e - \mathbb{I}_d) -2\log(\det(F_e))\right)+
 \frac{K}{2}\big(\det(F_e)-H\big)^2  
\end{align}
with bulk modulus $G=G(\psi)$ and phase volume $H=H(\psi)$ and inverse compressibility $K\in\mathbb{R}$. The remaining part of the free energy is 
\begin{align}
    W_\text{phase}(\psi,\nabla\psi,F) &= \left[\frac{1}{2}\nabla\psi\cdot \sigma\nabla\psi + W_\text{entropy}(\psi,F)\right]\det(F)
\end{align}
with second-order tensor $\sigma=\sigma(\psi,F)\in\mathbb{R}^{d\times d}$. 
Together with a given dissipation potential $R(q,\dot{q})$, different parameters, double-well or Flory-Huggins-type entropy $W_\text{entropy}$ and possible constraints $C(q,\nabla q)=0\in\mathbb{R}^M$ added through the Lagrangian in terms of $q_\lambda=(q,\lambda)$ with the optional Lagrange multiplier $\lambda:\Omega\to\mathbb{R}^M$
\begin{align}
\mathscr{L}(q_\lambda)=\mathscr{F}(q)+\int_\Omega C(q,\nabla q)\cdot\lambda\,{\rm d}x
\end{align}
we consider the formal gradient flow evolution $\partial_t q =-\nabla_R \mathscr{F}(q)$, which we solve by a saddle-point problem generated by the minimization problem
\begin{align}
%\partial_t q =-\nabla_R \mathscr{L}(q)
%\quad\Leftrightarrow\quad \left(
  \min_{v_\lambda=(v,\hat{\lambda})} \Big[R(q,v)+\langle \mathrm{D}\mathscr{L}(q_\lambda),v_\lambda\rangle\Big]%\right)
\end{align}
solved by incremental minimization as described in more detail in the manuscript.

# Example 3:

For the simplest example we consider a fixed two-dimensional domain $\Omega=(0,1)^2$ with homogeneous Dirichlet boundary conditions $u=0$ on $\partial\Omega$. We use one species $N=1$ and other constitutive relations / paramters

\begin{align}
G_1&=1, G_2= 0.3, G_3=0.3 \\
K&=0\\
% H&=0\\
\sigma&=\varepsilon F^{-1}F^{-\top} \qquad \varepsilon=0.02\\
F_p&=\mathbb{I}_2 \\ %(1+\alpha\psi)\mathbb{I}_2 \qquad \alpha=0.5 \\
W_\text{entropy}&=\frac{1}{2\varepsilon}(1-\psi_i^2)^2\\
R(q,\nabla q)&=\text{Cahn-Hilliard-type }R_\psi\text{ with }\mu=1 \text{  } \\
C(q,\nabla q)&=\text{no constraints in this example}
\end{align}

In [4]:
from fenics import *
from matplotlib import pyplot as plt
import numpy as np
from mshr import *

mesh  = Mesh('/content/gradient_flows_order_parameters_mechanics/colab/Data/mesh.xml')
mesh1 = Mesh('/content/gradient_flows_order_parameters_mechanics/colab/Data/mesh1.xml')

Vu_solid      = VectorElement("P", mesh1.ufl_cell(), 1)
V_solid       = FunctionSpace(mesh1,Vu_solid)

Vu    = VectorElement("P", mesh.ufl_cell(), 1) # V_u   = displacements
Vpsi  = FiniteElement("P", mesh.ufl_cell(), 1) # V_psi = order-parameter(s)
Upsi  = FiniteElement("P", mesh.ufl_cell(), 1) # U     = forces
R     = FiniteElement("P", mesh.ufl_cell(), 1) # Lagrange Multiplier
VxUxR = FunctionSpace(mesh, MixedElement([Vu,Vpsi,Vpsi,Upsi,Upsi,R]))

# homogeneous Dirichlet boundary conditions 
bc = [DirichletBC(VxUxR.sub(0),  Constant((0, 0)), 'on_boundary')]

def GshearF(psi1,psi2):
    psi3 = -1-psi1-psi2
    xi1 = (1+psi1)/2
    xi2 = (1+psi2)/2
    xi3 = (1+psi3)/2
    return  (xi1*G1 + xi2*G2 + xi3*G3)

In [8]:
print('Cahn Hilliard mobility: ', m)

Cahn Hilliard mobility:  1e-06


In [5]:
def incremental_minimization(old_q, dt ):
    
    q  = Function(VxUxR)   
    dq = TestFunction(VxUxR)

    v, psi1, psi2, feta1, feta2, lambda1      = split(q)      # Current solution
    old_v, old_psi1, old_psi2,*_              = split(old_q)  # Old solution
    dv,dpsi1,dpsi2,dfeta1,dfeta2,_            = split(dq)     # Test functions

    # Define elasticity stuff
    d       = psi1.geometric_dimension() 
    I       = Identity(d)
    F       = grad(v) + I       # strain
    old_F   = grad(old_v) + I   
    J       = det(F)            # Jacobian
    C       = F.T*F           # (right) Cauchy-Green tensor    
    
    psi3 = -1-psi1-psi2
    
    gradpsi1 = inv(F).T*grad(psi1)  
    gradpsi2 = inv(F).T*grad(psi2) 
    gradpsi3 = inv(F).T*grad(psi3) 

    # Stored strain energy density (compressible neo-Hookean model)
    e_elastic   = GshearF(psi1,psi2)/2*tr(C - I) 

    e_phase     = gamma1*( 1/(4*eps)*(1-(psi1)**2)**2 + (eps/2)*inner(gradpsi1,gradpsi1) )
    e_phase    += gamma2*( 1/(4*eps)*(1-(psi2)**2)**2 + (eps/2)*inner(gradpsi2,gradpsi2) )
    e_phase    += gamma3*( 1/(4*eps)*(1-(psi3)**2)**2 + (eps/2)*inner(gradpsi3,gradpsi3) )

    E   = (e_elastic + e_phase )*dx
    L   =  E + lambda1*(J-1)*dx
    
    # Finite difference time derivatives
    dot_v    = (v-old_v)/dt
    dot_psi1 = (psi1-old_psi1)
    dot_psi2 = (psi2-old_psi2)
    
    # Build residual:
    Res = derivative(L, q, dq) - inner(feta1,dpsi1)*dx - inner(feta2,dpsi2)*dx
    
    # Evolution 
    Res += inner(2*mu*sym(grad(dot_v)) , sym(grad(dv))  )*dx  
    Res += inner(dot_psi1, dfeta1)*dx   + dt*inner(m*inv(old_F).T*grad(feta1), inv(old_F).T*grad(dfeta1) )*dx 
    Res += inner(dot_psi2, dfeta2)*dx   + dt*inner(m*inv(old_F).T*grad(feta2), inv(old_F).T*grad(dfeta2) )*dx
      
    # Solve
    q.assign(old_q)  
    solve(Res==0,q,bc)
    E_free  = assemble(E)

    return q,E_free

The function *mesh_space* is used to refine the mesh at the interfaces. 

In [6]:
initial = Expression(("0",
                      "0",
                      "tanh(ff*(h0-x[1])/eps)",
                      "-1 + 2*(1+tanh(ff*(x[1]/h0-h0)/eps))*(1+tanh(ff/eps*(h1-pow(pow((x[0])/h0-H1/2,2.0)+pow(x[1]/h0-h0,2.0),0.5))))/4",
                      "0","0","0"), eps=eps,h0=h0,h1=h1,ff=interface_factor,H1=H1,degree=2)

old_q = interpolate(initial, VxUxR)

In [7]:
# Refine the mesh
bc, VxUxR, Vpsi, Vu, mesh   = refine_mesh(mesh,VxUxR,Vpsi,old_q)
old_q                       = interpolate(initial, VxUxR)
bc, VxUxR, Vpsi, Vu, mesh   = refine_mesh(mesh,VxUxR,Vpsi,old_q)
old_q                       = interpolate(initial, VxUxR)
bc, VxUxR, Vpsi, Vu, mesh   = refine_mesh(mesh,VxUxR,Vpsi,old_q)
old_q                       = interpolate(initial, VxUxR)
bc, VxUxR, Vpsi, Vu, mesh   = refine_mesh(mesh,VxUxR,Vpsi,old_q)
old_q                       = interpolate(initial, VxUxR)

In [ ]:
t         = 0
n_steps   = 300
dt        = 1e-3
energies  = []

output_mesh(mesh,filepath)
output_mesh1(mesh1,filepath)
output_solution(old_q,0,mesh,Vu,Vpsi,filepath,t)


out   = display(progress(0, n_steps), display_id=True)
for n in range(n_steps):
  dt_adapt = dt
  if (n<10) :
    dt_adapt = dt/10
  if (n>9) :
    dt_adapt = dt 

  t += dt
  q,E_free = incremental_minimization(old_q, dt)  
  out.update(progress(n+1,n_steps))
  energies.append([n,t,E_free])
  output_solution(q,n+1,mesh,Vu,Vpsi,filepath,t)
  old_q.assign(q)

E = list(zip(*energies))
np.save(filepath + "energies",E)